In [ ]:
import pandas as pd
import json
import time
import os
import asyncio
import aiofiles
from pathlib import Path
import signal
import sys
from datetime import datetime
import re
import numpy as np
from scipy import stats
from collections import defaultdict
from google import genai
import random
from typing import List, Dict, Optional, Tuple
import logging
import pickle
from dataclasses import dataclass, asdict
from datasets import load_dataset

# Configure logging with UTF-8 encoding to handle Unicode characters
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('jeebench_evaluation.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class EvaluationState:
    """Persistent state for resuming evaluation"""
    current_run: int
    completed_questions: int
    total_questions: int
    all_run_summaries: List[Dict]
    failed_questions: List[Dict]
    start_time: float
    last_save_time: float

class DistributedGeminiClient:
    """Truly distributed Gemini API client across multiple keys"""
    
    def __init__(self, api_keys: List[str]):
        self.api_keys = api_keys
        
        # Validate API keys
        valid_clients = []
        valid_keys = []
        
        for i, key in enumerate(api_keys):
            if key and key != "YOUR_API_KEY_1" and len(key) > 10:  # Basic validation
                try:
                    client = genai.Client(api_key=key)
                    valid_clients.append(client)
                    valid_keys.append(key)
                    logger.info(f"API Key {i+1}: Valid ✓")
                except Exception as e:
                    logger.error(f"API Key {i+1}: Invalid - {e}")
            else:
                logger.warning(f"API Key {i+1}: Skipped (placeholder or empty)")
        
        if not valid_clients:
            raise ValueError("No valid API keys provided!")
        
        self.clients = valid_clients
        self.api_keys = valid_keys
        self.request_times = [[] for _ in valid_keys]
        self.max_requests_per_minute = 25
        
        logger.info(f"Initialized {len(valid_keys)} valid distributed Gemini clients")
    
    def _clean_old_requests(self, client_idx: int):
        """Remove request times older than 1 minute"""
        now = time.time()
        self.request_times[client_idx] = [
            req_time for req_time in self.request_times[client_idx] 
            if now - req_time < 60
        ]
    
    async def _wait_for_rate_limit(self, client_idx: int):
        """Wait if approaching rate limits for specific client"""
        self._clean_old_requests(client_idx)
        
        if len(self.request_times[client_idx]) >= self.max_requests_per_minute:
            oldest_request = min(self.request_times[client_idx])
            wait_time = 60 - (time.time() - oldest_request) + 1
            
            if wait_time > 0:
                logger.info(f"Rate limit for client {client_idx}, waiting {wait_time:.1f}s")
                await asyncio.sleep(wait_time)
    
    async def generate_content_distributed(self, client_idx: int, prompt: str, max_retries: int = 3) -> Optional[str]:
        """Generate content using specific client with rate limiting"""
        
        await self._wait_for_rate_limit(client_idx)
        
        for attempt in range(max_retries):
            try:
                # Record request time
                self.request_times[client_idx].append(time.time())
                
                client = self.clients[client_idx]
                
                response = await asyncio.to_thread(
                    client.models.generate_content,
                    model="gemma-3-27b-it",
                    contents=[prompt]
                )
                
                return response.text
                
            except Exception as e:
                if "rate limit" in str(e).lower() or "quota" in str(e).lower():
                    wait_time = (2 ** attempt) * 60
                    logger.warning(f"Rate limit on client {client_idx}, attempt {attempt + 1}, waiting {wait_time}s")
                    await asyncio.sleep(wait_time)
                else:
                    logger.error(f"Client {client_idx} error on attempt {attempt + 1}: {e}")
                    if attempt == max_retries - 1:
                        return None
                    await asyncio.sleep(10)
        
        return None

class JEEBenchGemma3Evaluator:
    def __init__(self, api_keys: List[str], num_runs: int = 10):
        self.num_runs = num_runs
        
        # Load JEEBench dataset
        logger.info("Loading JEEBench dataset...")
        self.dataset = load_dataset("daman1209arora/jeebench")
        
        # Convert to pandas DataFrame for easier manipulation
        self.df = pd.DataFrame(self.dataset['test'])
        logger.info(f"Loaded JEEBench dataset with {len(self.df)} questions")
        
        # Print dataset info
        logger.info(f"Dataset columns: {list(self.df.columns)}")
        logger.info(f"Question types: {self.df['type'].value_counts().to_dict()}")
        logger.info(f"Subjects: {self.df['subject'].value_counts().to_dict()}")
        
        # Initialize distributed Gemini client
        self.client = DistributedGeminiClient(api_keys)
        
        # Results and state management
        self.results_dir = Path("jeebench_evaluation_results")
        self.results_dir.mkdir(exist_ok=True)
        
        self.state_file = self.results_dir / "evaluation_state.pkl"
        self.state = self.load_or_create_state()
        
        # Control flags
        self.stop_requested = False
        self.interrupted = False
        
        logger.info(f"Results will be saved to: {self.results_dir}")
        logger.info(f"Running {num_runs} evaluations with {len(api_keys)} API keys")
    
    def load_or_create_state(self) -> EvaluationState:
        """Load existing state or create new one"""
        if self.state_file.exists():
            try:
                with open(self.state_file, 'rb') as f:
                    state = pickle.load(f)
                logger.info(f"Resumed from Run {state.current_run}, Question {state.completed_questions}/{state.total_questions}")
                return state
            except Exception as e:
                logger.error(f"Error loading state: {e}")
        
        # Create new state
        return EvaluationState(
            current_run=1,
            completed_questions=0,
            total_questions=len(self.df) * self.num_runs,
            all_run_summaries=[],
            failed_questions=[],
            start_time=time.time(),
            last_save_time=time.time()
        )
    
    async def save_state(self):
        """Save current evaluation state"""
        self.state.last_save_time = time.time()
        try:
            # Use temporary file for atomic write
            temp_file = self.state_file.with_suffix('.tmp')
            with open(temp_file, 'wb') as f:
                pickle.dump(self.state, f)
            temp_file.replace(self.state_file)
            
            logger.info(f"State saved: Run {self.state.current_run}, {self.state.completed_questions}/{self.state.total_questions} questions")
        except Exception as e:
            logger.error(f"Error saving state: {e}")
    
    def create_question_prompt(self, question_data: pd.Series) -> str:
        """Create appropriate prompt based on question type"""
        question_type = question_data['type']
        question_text = question_data['question']
        
        # Base prompt with question
        prompt = f"Question: {question_text}\n\n"
        
        # Add type-specific instructions
        if question_type == "MCQ":
            prompt += """This is a multiple choice question. Please analyze the question carefully, reason step-by-step and provide your answer.

For this question:
- Choose exactly ONE option (A, B, C, or D)
- Format your answer in \\boxed{} as just one letter (e.g., \\boxed{A})"""
        elif question_type == "MCQ(multiple)":
            prompt += """This is a multiple choice question where multiple options can be correct. Please analyze the question carefully, reason step-by-step and provide your answer.

For this question:
- Choose ONE OR MORE options (A, B, C, and/or D)
- Format your answer in \\boxed{} with letters (e.g., \\boxed{ABC} or \\boxed{B})"""
        elif question_type == "Integer":
            prompt += """This is a numerical question. Please analyze the question carefully, reason step-by-step and provide your answer.

For this question:
- Provide a numerical value
- Round to appropriate decimal places if needed
- Format your answer in \\boxed{} (e.g., \\boxed{2.5} or \\boxed{42})"""
        else:
            # Default case
            prompt += """Please analyze the question carefully, reason step-by-step and provide your answer.
Format your answer in \\boxed{} (e.g., \\boxed{A} for MCQ or \\boxed{42} for numerical)"""
        
        return prompt
    
    def extract_answer(self, response_text: str, question_type: str) -> str:
        """Extract the final answer from model response"""
        try:
            # First try to find boxed answer
            boxed_match = re.search(r'\\boxed\{([^}]+)\}', response_text)
            if boxed_match:
                answer = boxed_match.group(1).strip()
            else:
                # Try other patterns
                answer_patterns = [
                    r'\*\*Answer:\*\*\s*(.+)',
                    r'Answer:\s*(.+)',
                    r'Final answer:\s*(.+)',
                ]
                answer = None
                for pattern in answer_patterns:
                    match = re.search(pattern, response_text, re.IGNORECASE)
                    if match:
                        answer = match.group(1).strip()
                        break
                if not answer:
                    # Take last line as fallback
                    lines = response_text.strip().split('\n')
                    answer = lines[-1].strip()
            
            # Clean up answer based on question type
            if question_type == "MCQ":
                # Extract single letter
                match = re.search(r'[ABCD]', answer.upper())
                return match.group(0) if match else answer[:10]
            elif question_type == "MCQ(multiple)":
                # Extract multiple letters
                letters = re.findall(r'[ABCD]', answer.upper())
                unique_letters = sorted(set(letters))
                return ''.join(unique_letters) if unique_letters else answer[:20]
            elif question_type == "Integer":
                # Extract number
                number_match = re.search(r'-?\d+\.?\d*', answer)
                return number_match.group(0) if number_match else answer[:20]
            
            return answer[:50]
        except Exception as e:
            logger.error(f"Error extracting answer: {e}")
            return response_text[:50]
    
    def is_answer_correct(self, predicted_answer: str, question_data: pd.Series) -> bool:
        """Check if predicted answer is correct"""
        try:
            predicted = str(predicted_answer).strip().upper()
            correct = str(question_data['gold']).strip().upper()
            question_type = question_data['type']
            
            if question_type == "MCQ":
                return predicted == correct
            elif question_type == "MCQ(multiple)":
                # Handle multiple choice with multiple correct answers
                pred_letters = set(re.findall(r'[ABCD]', predicted))
                correct_letters = set(re.findall(r'[ABCD]', correct))
                return pred_letters == correct_letters
            elif question_type == "Integer":
                # First try exact match
                if predicted == correct:
                    return True
                # Try numerical comparison with tolerance
                try:
                    pred_num = float(predicted)
                    correct_num = float(correct)
                    tolerance = abs(correct_num) * 0.01 if abs(correct_num) > 1 else 0.01
                    return abs(pred_num - correct_num) <= tolerance
                except ValueError:
                    return predicted == correct
            
            # Default exact match
            return predicted == correct
        except Exception as e:
            logger.error(f"Error comparing answers: {e}")
            return False
    
    async def evaluate_single_question(self, client_idx: int, question_data: pd.Series, run_id: int, question_idx: int) -> Optional[Dict]:
        """Evaluate a single question using specific client"""
        try:
            prompt = self.create_question_prompt(question_data)
            
            start_time = time.time()
            response_text = await self.client.generate_content_distributed(
                client_idx, prompt
            )
            inference_time = time.time() - start_time
            
            if not response_text:
                logger.error(f"Failed to get response for question {question_data.get('index', question_idx)}")
                return None
            
            predicted_answer = self.extract_answer(response_text, question_data['type'])
            is_correct = self.is_answer_correct(predicted_answer, question_data)
            
            # Log each completion
            status = "[OK]" if is_correct else "[FAIL]"
            logger.info(f"Client {client_idx} | Run {run_id} | Q{question_idx+1}: {status} ({inference_time:.1f}s)")
            
            return {
                'run_id': run_id,
                'question_idx': question_idx,
                'client_idx': client_idx,
                'dataset_index': question_data.get('index', question_idx),
                'subject': question_data['subject'],
                'question_type': question_data['type'],
                'question_text': question_data['question'][:200] + "..." if len(question_data['question']) > 200 else question_data['question'],
                'correct_answer': question_data['gold'],
                'predicted_answer': predicted_answer,
                'is_correct': is_correct,
                'inference_time': inference_time,
                'full_response': response_text[:1000] + "..." if len(response_text) > 1000 else response_text  # Truncate for storage
            }
        except Exception as e:
            logger.error(f"Error evaluating question {question_data.get('index', question_idx)}: {e}")
            return None
    
    async def process_questions_parallel(self, questions_with_indices: List[Tuple[int, pd.Series]], run_id: int) -> List[Dict]:
        """Process questions in parallel across multiple API keys - SIMPLE APPROACH"""
        
        num_clients = len(self.client.clients)
        
        # Distribute questions across clients (round-robin)
        client_questions = [[] for _ in range(num_clients)]
        for i, question_item in enumerate(questions_with_indices):
            client_idx = i % num_clients
            client_questions[client_idx].append(question_item)
        
        # Log distribution
        for client_idx, questions in enumerate(client_questions):
            logger.info(f"Client {client_idx}: {len(questions)} questions assigned")
        
        # Process function for each client
        async def process_client_batch(client_idx: int, assigned_questions: List[Tuple[int, pd.Series]]) -> List[Dict]:
            # Force immediate yield to allow other tasks to start
            await asyncio.sleep(0.01 * client_idx)  # Stagger slightly
    
            logger.info(f"🚀 Client {client_idx} starting {len(assigned_questions)} questions")
            
            client_results = []
            for i, (question_idx, question_data) in enumerate(assigned_questions):
                try:
                    result = await self.evaluate_single_question(
                        client_idx, question_data, run_id, question_idx
                    )
                    if result:
                        client_results.append(result)
                    
                    # Progress every 25 questions
                    if (i + 1) % 25 == 0:
                        logger.info(f"📊 Client {client_idx}: {i+1}/{len(assigned_questions)} done")
                        
                except Exception as e:
                    logger.error(f"❌ Client {client_idx} error: {e}")
                    continue
            
            logger.info(f"✅ Client {client_idx} finished: {len(client_results)}/{len(assigned_questions)} successful")
            return client_results
        
        # Create tasks for all clients
        tasks = []
        for client_idx in range(num_clients):
            if client_questions[client_idx]:  # Only if client has questions
                task = asyncio.create_task(
                    process_client_batch(client_idx, client_questions[client_idx])
                )
                tasks.append(task)
                logger.info(f"✨ Task created for Client {client_idx}")
        
        logger.info(f"🎯 Running {len(tasks)} client tasks in parallel...")
        
        # Wait for all tasks to complete
        all_results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Combine results
        combined_results = []
        for i, client_result in enumerate(all_results):
            if isinstance(client_result, Exception):
                logger.error(f"Client {i} task failed: {client_result}")
            elif isinstance(client_result, list):
                combined_results.extend(client_result)
        
        logger.info(f"🏁 Total results: {len(combined_results)}")
        return combined_results
    
    async def run_single_evaluation_run(self, run_id: int) -> Optional[Dict]:
        """Run a single evaluation run with parallel processing"""
        logger.info(f"\n{'='*60}")
        logger.info(f"Starting Run {run_id}/{self.num_runs}")
        logger.info(f"{'='*60}")
        
        # Shuffle questions for this run
        shuffled_df = self.df.sample(frac=1, random_state=run_id).reset_index(drop=True)
        
        # Process all questions in parallel
        questions_with_indices = [(idx, row) for idx, row in shuffled_df.iterrows()]
        
        run_start_time = time.time()
        results = await self.process_questions_parallel(questions_with_indices, run_id)
        run_duration = time.time() - run_start_time
        
        if not results:
            logger.error(f"No valid results for run {run_id}")
            return None
        
        # Calculate accuracy
        correct_count = sum(1 for r in results if r['is_correct'])
        accuracy = (correct_count / len(results)) * 100
        
        # Create run summary
        run_summary = {
            'run_id': run_id,
            'total_questions': len(results),
            'correct_answers': correct_count,
            'accuracy': accuracy,
            'duration': run_duration,
            'avg_time_per_question': run_duration / len(results),
            'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
            'results': results
        }
        
        logger.info(f"Run {run_id} completed: {accuracy:.2f}% accuracy ({correct_count}/{len(results)}) in {run_duration:.1f}s")
        logger.info(f"Average time per question: {run_summary['avg_time_per_question']:.1f}s")
        
        return run_summary
    
    async def save_run_results(self, run_summary: Dict):
        """Save results for a single run"""
        timestamp = run_summary['timestamp']
        filename = f"jeebench_gemma3_run_{run_summary['run_id']:02d}_{timestamp}.json"
        filepath = self.results_dir / filename
        
        async with aiofiles.open(filepath, 'w', encoding='utf-8') as f:
            await f.write(json.dumps(run_summary, indent=2, ensure_ascii=False))
        
        logger.info(f"Run {run_summary['run_id']} results saved to: {filename}")
    
    def calculate_statistics(self, all_run_summaries: List[Dict]) -> Dict:
        """Calculate overall statistics across all runs"""
        accuracies = [summary['accuracy'] for summary in all_run_summaries]
        
        return {
            'num_runs': len(accuracies),
            'mean_accuracy': np.mean(accuracies),
            'std_accuracy': np.std(accuracies, ddof=1),
            'sem_accuracy': stats.sem(accuracies),
            'min_accuracy': np.min(accuracies),
            'max_accuracy': np.max(accuracies),
            'confidence_interval_95': stats.t.interval(
                0.95, len(accuracies) - 1,
                loc=np.mean(accuracies),
                scale=stats.sem(accuracies)
            ) if len(accuracies) > 1 else (np.mean(accuracies), np.mean(accuracies)),
            'individual_accuracies': accuracies
        }
    
    def analyze_convergence_and_variance(self, all_run_summaries: List[Dict]) -> Dict:
        """Analyze convergence for optimal k determination"""
        accuracies = [summary['accuracy'] for summary in all_run_summaries]
        n_runs = len(accuracies)
        
        convergence_analysis = {
            'k_values': [],
            'running_means': [],
            'running_stds': [],
            'running_sems': [],
            'confidence_intervals': [],
            'relative_changes': [],
            'stability_metrics': [],
            'cost_effectiveness': []
        }
        
        for k in range(3, n_runs + 1):
            subset_accuracies = accuracies[:k]
            
            mean_acc = np.mean(subset_accuracies)
            std_acc = np.std(subset_accuracies, ddof=1)
            sem_acc = stats.sem(subset_accuracies)
            
            if k > 1:
                ci = stats.t.interval(0.95, k-1, loc=mean_acc, scale=sem_acc)
                ci_width = ci[1] - ci[0]
            else:
                ci = (mean_acc, mean_acc)
                ci_width = 0
            
            if k > 3:
                prev_mean = convergence_analysis['running_means'][-1]
                relative_change = abs(mean_acc - prev_mean) / prev_mean * 100
            else:
                relative_change = np.inf
            
            cv = std_acc / mean_acc * 100 if mean_acc > 0 else np.inf
            cost_effectiveness = ci_width * k
            
            convergence_analysis['k_values'].append(k)
            convergence_analysis['running_means'].append(mean_acc)
            convergence_analysis['running_stds'].append(std_acc)
            convergence_analysis['running_sems'].append(sem_acc)
            convergence_analysis['confidence_intervals'].append(ci)
            convergence_analysis['relative_changes'].append(relative_change)
            convergence_analysis['stability_metrics'].append(cv)
            convergence_analysis['cost_effectiveness'].append(cost_effectiveness)
        
        optimal_k_analysis = self._find_optimal_k(convergence_analysis)
        convergence_analysis['optimal_k_recommendations'] = optimal_k_analysis
        
        return convergence_analysis
    
    def _find_optimal_k(self, convergence_data: Dict) -> Dict:
        """Find optimal k using multiple criteria"""
        k_values = convergence_data['k_values']
        relative_changes = convergence_data['relative_changes']
        confidence_intervals = convergence_data['confidence_intervals']
        
        recommendations = {}
        
        # Convergence threshold
        stable_change_threshold = 2.0
        for i, (k, rel_change) in enumerate(zip(k_values, relative_changes)):
            if rel_change < stable_change_threshold and k >= 5:
                recommendations['convergence_threshold'] = {
                    'k': k,
                    'reason': f'First k where relative change < {stable_change_threshold}%',
                    'relative_change': rel_change
                }
                break
        
        # CI width thresholds
        target_ci_widths = [1.0, 2.0, 3.0, 5.0]
        for target_width in target_ci_widths:
            for i, (k, ci) in enumerate(zip(k_values, confidence_intervals)):
                ci_width = ci[1] - ci[0]
                if ci_width <= target_width and k >= 5:
                    recommendations[f'ci_width_{target_width}'] = {
                        'k': k,
                        'reason': f'First k achieving CI width ≤ {target_width}%',
                        'ci_width': ci_width
                    }
                    break
        
        # Pragmatic minimum
        for i, (k, ci) in enumerate(zip(k_values, confidence_intervals)):
            ci_width = ci[1] - ci[0]
            if k >= 10 and ci_width <= 4.0:
                recommendations['pragmatic_minimum'] = {
                    'k': k,
                    'reason': 'Pragmatic balance: k≥10 with CI width ≤4%',
                    'ci_width': ci_width
                }
                break
        
        return recommendations
    
    async def run_evaluation(self):
        """Run complete evaluation with resume capability"""
        logger.info("Starting JEEBench Gemma-3-27B Evaluation with Resume Capability")
        logger.info(f"Dataset: {len(self.df)} questions")
        logger.info(f"API Keys: {len(self.client.clients)}")
        logger.info(f"Total runs planned: {self.num_runs}")
        
        try:
            # Resume from where we left off
            for run_id in range(self.state.current_run, self.num_runs + 1):
                if self.stop_requested:
                    break
                
                # Run single evaluation
                run_summary = await self.run_single_evaluation_run(run_id)
                
                if run_summary:
                    # Save run results
                    await self.save_run_results(run_summary)
                    
                    # Update state
                    self.state.all_run_summaries.append(run_summary)
                    self.state.current_run = run_id + 1
                    self.state.completed_questions += len(run_summary['results'])
                    
                    # Save state after each run
                    await self.save_state()
                    
                    # Print progress
                    progress = (run_id / self.num_runs) * 100
                    elapsed = time.time() - self.state.start_time
                    eta = (elapsed / run_id) * (self.num_runs - run_id) if run_id > 0 else 0
                    
                    logger.info(f"Progress: {progress:.1f}% | ETA: {eta/3600:.1f}h | Avg accuracy so far: {np.mean([s['accuracy'] for s in self.state.all_run_summaries]):.2f}%")
        
        except KeyboardInterrupt:
            logger.info("Evaluation interrupted by user")
            self.interrupted = True
        except Exception as e:
            logger.error(f"Error during evaluation: {e}")
        finally:
            # Always save final state
            await self.save_state()
            
            # Generate final report if we have results
            if self.state.all_run_summaries:
                await self.generate_final_report()
        
        logger.info("Evaluation session ended!")
    
    async def generate_final_report(self):
        """Generate comprehensive final report"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        report_file = self.results_dir / f"jeebench_gemma3_final_report_{timestamp}.txt"
        
        stats = self.calculate_statistics(self.state.all_run_summaries)
        convergence_data = self.analyze_convergence_and_variance(self.state.all_run_summaries)
        
        # Collect all results for detailed analysis
        all_results = []
        for run_summary in self.state.all_run_summaries:
            all_results.extend(run_summary['results'])
        
        report_content = f"""JEEBench Gemma-3-27B Evaluation Report
{'='*80}

Evaluation Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Model: Gemma-3-27B (via Gemini API)
Dataset: JEEBench ({len(self.df)} questions)
Completed Runs: {len(self.state.all_run_summaries)}/{self.num_runs}
API Keys Used: {len(self.client.clients)}

PERFORMANCE SUMMARY
{'-'*40}
Mean Accuracy: {stats['mean_accuracy']:.2f}% ± {stats['std_accuracy']:.2f}%
Standard Error: {stats['sem_accuracy']:.2f}%
95% Confidence Interval: [{stats['confidence_interval_95'][0]:.2f}%, {stats['confidence_interval_95'][1]:.2f}%]
Range: {stats['min_accuracy']:.2f}% - {stats['max_accuracy']:.2f}%

OPTIMAL k RECOMMENDATIONS FOR COST-EFFECTIVE EVALUATION
{'-'*60}
"""
        
        # Add recommendations
        recommendations = convergence_data['optimal_k_recommendations']
        priority_recs = ['pragmatic_minimum', 'convergence_threshold', 'ci_width_2.0']
        
        for rec_name in priority_recs:
            if rec_name in recommendations:
                rec = recommendations[rec_name]
                report_content += f"{rec_name.replace('_', ' ').title()}: k = {rec['k']} ({rec['reason']})\n"
        
        report_content += f"\nPERFORMANCE BREAKDOWN\n{'-'*40}\n"
        
        # Performance by category
        categories = ['subject', 'question_type']
        for category in categories:
            report_content += f"\nBy {category.title()}:\n"
            category_stats = defaultdict(lambda: {'correct': 0, 'total': 0})
            
            for result in all_results:
                cat_value = result[category]
                category_stats[cat_value]['total'] += 1
                if result['is_correct']:
                    category_stats[cat_value]['correct'] += 1
            
            for cat_value, stats_dict in sorted(category_stats.items()):
                accuracy = (stats_dict['correct'] / stats_dict['total']) * 100
                report_content += f"  {cat_value}: {accuracy:.2f}% ({stats_dict['correct']}/{stats_dict['total']})\n"
        
        # Add individual run accuracies
        report_content += f"\nINDIVIDUAL RUN ACCURACIES\n{'-'*40}\n"
        for i, accuracy in enumerate(stats['individual_accuracies'], 1):
            report_content += f"Run {i}: {accuracy:.2f}%\n"
        
        # Add convergence analysis summary
        if len(convergence_data['k_values']) > 5:
            report_content += f"\nCONVERGENCE ANALYSIS\n{'-'*40}\n"
            report_content += f"For reliable estimates (±2% CI width): k ≥ {convergence_data['optimal_k_recommendations'].get('ci_width_2.0', {}).get('k', 'N/A')}\n"
            report_content += f"For stable convergence (<2% change): k ≥ {convergence_data['optimal_k_recommendations'].get('convergence_threshold', {}).get('k', 'N/A')}\n"
        
        async with aiofiles.open(report_file, 'w', encoding='utf-8') as f:
            await f.write(report_content)
        
        logger.info(f"Final report saved to: {report_file}")
        logger.info(f"Pass@1 Accuracy: {stats['mean_accuracy']:.2f}% ± {stats['std_accuracy']:.2f}%")

# Configuration settings
API_KEYS = [
    "API_KEY_1",
    "API_KEY_2", 
    "API_KEY_3",
    "API_KEY_4",
    "API_KEY_5"
]

NUM_RUNS = 10

async def run_evaluation():
    """Main evaluation function for Jupyter"""
    evaluator = JEEBenchGemma3Evaluator(API_KEYS, NUM_RUNS)
    await evaluator.run_evaluation()

async def resume_evaluation():
    """Resume evaluation from saved state"""
    evaluator = JEEBenchGemma3Evaluator(API_KEYS, NUM_RUNS)
    logger.info("Resuming evaluation from saved state...")
    await evaluator.run_evaluation()

async def check_progress():
    """Check current progress without running evaluation"""
    evaluator = JEEBenchGemma3Evaluator(API_KEYS, NUM_RUNS)
    state = evaluator.state
    
    print(f"\n{'='*60}")
    print("CURRENT EVALUATION PROGRESS")
    print(f"{'='*60}")
    print(f"Current Run: {state.current_run}/{NUM_RUNS}")
    print(f"Completed Questions: {state.completed_questions:,}/{state.total_questions:,}")
    print(f"Progress: {(state.completed_questions/state.total_questions)*100:.1f}%")
    
    if state.all_run_summaries:
        accuracies = [s['accuracy'] for s in state.all_run_summaries]
        print(f"Completed Runs: {len(state.all_run_summaries)}")
        print(f"Average Accuracy: {np.mean(accuracies):.2f}% ± {np.std(accuracies, ddof=1):.2f}%")
        
        elapsed = time.time() - state.start_time
        if len(state.all_run_summaries) > 0:
            eta = (elapsed / len(state.all_run_summaries)) * (NUM_RUNS - len(state.all_run_summaries))
            print(f"Elapsed Time: {elapsed/3600:.1f}h")
            print(f"Estimated Time Remaining: {eta/3600:.1f}h")
    
    print(f"Last Save: {datetime.fromtimestamp(state.last_save_time).strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"State File: {evaluator.state_file}")
    print(f"{'='*60}\n")

async def reset_evaluation():
    """Reset evaluation state (use with caution!)"""
    evaluator = JEEBenchGemma3Evaluator(API_KEYS, NUM_RUNS)
    
    print("⚠️  WARNING: This will delete all progress and start fresh!")
    confirm = input("Type 'RESET' to confirm: ")
    
    if confirm == "RESET":
        if evaluator.state_file.exists():
            evaluator.state_file.unlink()
            print("✅ Evaluation state reset successfully!")
        else:
            print("ℹ️  No existing state file found.")
    else:
        print("❌ Reset cancelled.")

def load_jeebench_sample(n_samples: int = 5):
    """Load and display a sample of JEEBench questions for inspection"""
    from datasets import load_dataset
    
    print("Loading JEEBench dataset...")
    ds = load_dataset("daman1209arora/jeebench")
    df = pd.DataFrame(ds['test'])
    
    print(f"\nDataset Info:")
    print(f"Total questions: {len(df)}")
    print(f"Columns: {list(df.columns)}")
    print(f"Question types: {df['type'].value_counts().to_dict()}")
    print(f"Subjects: {df['subject'].value_counts().to_dict()}")
    
    print(f"\nSample {n_samples} questions:")
    print("="*80)
    
    sample_df = df.sample(n=min(n_samples, len(df)), random_state=42)
    
    for idx, row in sample_df.iterrows():
        print(f"\nQuestion {idx + 1}:")
        print(f"Subject: {row['subject']}")
        print(f"Type: {row['type']}")
        print(f"Question: {row['question'][:200]}{'...' if len(row['question']) > 200 else ''}")
        print(f"Gold Answer: {row['gold']}")
        print("-" * 60)
    
    return df

"""
# To inspect the dataset first:
df = load_jeebench_sample(10)

# To start new evaluation:
await run_evaluation()

# To resume from where you left off:
await resume_evaluation()

# To check current progress:
await check_progress()

# To reset everything (use carefully!):
await reset_evaluation()
"""

'\n# To inspect the dataset first:\ndf = load_jeebench_sample(10)\n\n# To start new evaluation:\nawait run_evaluation()\n\n# To resume from where you left off:\nawait resume_evaluation()\n\n# To check current progress:\nawait check_progress()\n\n# To reset everything (use carefully!):\nawait reset_evaluation()\n'

In [ ]:
await run_evaluation()

2025-07-27 16:08:13,968 - INFO - Loading JEEBench dataset...
2025-07-27 16:08:28,833 - INFO - Loaded JEEBench dataset with 515 questions
2025-07-27 16:08:28,835 - INFO - Dataset columns: ['subject', 'description', 'gold', 'index', 'type', 'question']
2025-07-27 16:08:28,836 - INFO - Question types: {'MCQ(multiple)': 186, 'Numeric': 137, 'MCQ': 110, 'Integer': 82}
2025-07-27 16:08:28,837 - INFO - Subjects: {'math': 236, 'chem': 156, 'phy': 123}
2025-07-27 16:08:29,532 - INFO - API Key 1: Valid ✓
2025-07-27 16:08:30,189 - INFO - API Key 2: Valid ✓
2025-07-27 16:08:30,837 - INFO - API Key 3: Valid ✓
2025-07-27 16:08:31,449 - INFO - API Key 4: Valid ✓
2025-07-27 16:08:32,079 - INFO - API Key 5: Valid ✓
2025-07-27 16:08:32,080 - INFO - Initialized 5 valid distributed Gemini clients
2025-07-27 16:08:32,081 - INFO - Results will be saved to: jeebench_evaluation_results
2025-07-27 16:08:32,081 - INFO - Running 10 evaluations with 5 API keys
2025-07-27 16:08:32,081 - INFO - Starting JEEBench Ge

In [ ]:
await resume_evaluation()

2025-07-28 01:21:40,702 - INFO - Loading JEEBench dataset...
2025-07-28 01:21:46,318 - INFO - Loaded JEEBench dataset with 515 questions
2025-07-28 01:21:46,320 - INFO - Dataset columns: ['subject', 'description', 'gold', 'index', 'type', 'question']
2025-07-28 01:21:46,321 - INFO - Question types: {'MCQ(multiple)': 186, 'Numeric': 137, 'MCQ': 110, 'Integer': 82}
2025-07-28 01:21:46,322 - INFO - Subjects: {'math': 236, 'chem': 156, 'phy': 123}
2025-07-28 01:21:46,966 - INFO - API Key 1: Valid ✓
2025-07-28 01:21:47,564 - INFO - API Key 2: Valid ✓
2025-07-28 01:21:48,158 - INFO - API Key 3: Valid ✓
2025-07-28 01:21:48,765 - INFO - API Key 4: Valid ✓
2025-07-28 01:21:49,506 - INFO - API Key 5: Valid ✓
2025-07-28 01:21:49,506 - INFO - Initialized 5 valid distributed Gemini clients
2025-07-28 01:21:49,710 - INFO - Resumed from Run 9, Question 4117/5150
2025-07-28 01:21:49,710 - INFO - Results will be saved to: jeebench_evaluation_results
2025-07-28 01:21:49,711 - INFO - Running 10 evaluatio

In [3]:
await resume_evaluation()

2025-07-28 02:08:16,461 - INFO - Loading JEEBench dataset...
2025-07-28 02:08:23,227 - INFO - Loaded JEEBench dataset with 515 questions
2025-07-28 02:08:23,228 - INFO - Dataset columns: ['subject', 'description', 'gold', 'index', 'type', 'question']
2025-07-28 02:08:23,229 - INFO - Question types: {'MCQ(multiple)': 186, 'Numeric': 137, 'MCQ': 110, 'Integer': 82}
2025-07-28 02:08:23,231 - INFO - Subjects: {'math': 236, 'chem': 156, 'phy': 123}
2025-07-28 02:08:23,817 - INFO - API Key 1: Valid ✓
2025-07-28 02:08:24,346 - INFO - API Key 2: Valid ✓
2025-07-28 02:08:24,876 - INFO - API Key 3: Valid ✓
2025-07-28 02:08:25,409 - INFO - API Key 4: Valid ✓
2025-07-28 02:08:25,943 - INFO - API Key 5: Valid ✓
2025-07-28 02:08:25,944 - INFO - Initialized 5 valid distributed Gemini clients
2025-07-28 02:08:26,104 - INFO - Resumed from Run 10, Question 4632/5150
2025-07-28 02:08:26,105 - INFO - Results will be saved to: jeebench_evaluation_results
2025-07-28 02:08:26,105 - INFO - Running 10 evaluati